In [17]:
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import data_import
from sklearn.cluster import KMeans
from data_preprocessing import KMeansDimensionalityReduction

Load data from the `npy` objects 

In [18]:
# Import data from Contagio dataset
X_tot, df_tot, X_train, X_test, y_train, y_test = data_import.import_data_train_test('Contagio')

Visualize the scatter matrix to see which features are correlated:

In [19]:
#scatter_matrix(df_tot, c=df_tot['malware'], figsize=(30,20))

In [22]:
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
pca = PCA(n_components=2)
X_test =  pca.fit(X_train).transform(X_test)
X_train = pca.fit(X_train).transform(X_train)
fig1 = px.scatter(x=X_train[y_train.ravel()==False, 0], y=X_train[y_train.ravel()==False, 1])
fig1.update_traces(marker=dict(color='green'))
fig2 = px.scatter(x=X_train[y_train.ravel()==True, 0], y=X_train[y_train.ravel()==True, 1])
fig2.update_traces(marker=dict(color='red'))
f = X_train[y_train.ravel()==False, 0].shape[0]
t = X_train[y_train.ravel()==True, 0].shape[0]
#plt.scatter(X_train[y_train.values.ravel()==False, 0], np.ones(f), alpha=0.05)
#plt.scatter(X_train[y_train.values.ravel()==True, 0], np.ones(t), alpha=0.05)
#px.xlim([-75, 0])
#px.ylim([-50, 50])
fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()

# Class separability analysis
$S_b = \frac{1}{N}\sum_{i=1}^{c}n_c(\mu_i-\mu)(\mu_i-\mu)^T$\
$S_w = frac{1}{N}\sum_{i=1}^{c}\sum_{j=1}^{n_c}\,(x_{ij}-\mu_i)(x_{ij}-\mu_i)^T$\
Property: $S_w + S_b = C$\
1. Class Scatter Matrices (CSM)\
$J = \frac{tr\{S_b\}}{tr\{S_w\}}$\
J is an unbounded measure. The larger the value
of J the smaller the within class scatter as
compared to the between class scatter.

In [ ]:
def computeSb(X, y):
    X_1 = X[y==True, :]
    X_0 = X[y==False, :]
    N_0 = X_0.shape[0]
    N_1 = X_1.shape[0]
    N = X.shape[0]
    mu_1 = np.mean(X_1, axis=0).reshape(-1,1)
    mu_0 = np.mean(X_0, axis=0).reshape(-1,1)
    mu = np.mean(X, axis=0).reshape(-1,1)
    return 1/N * (N_0 * np.dot(mu_0-mu, (mu_0-mu).T) + N_1 * np.dot(mu_1-mu, (mu_1-mu).T))

In [ ]:
def computeSw(X, y):
    X_1 = X[y==True, :]
    X_0 = X[y==False, :]
    N_0 = X_0.shape[0]
    N_1 = X_1.shape[0]
    N = X.shape[0]
    return 1/N * (N_0 * np.cov(X_0.T, bias=True) + N_1 * np.cov(X_1.T, bias=True))

In [ ]:
Sw = computeSw(X_train,y_train)
Sb = computeSb(X_train,y_train)
J = np.trace(Sb) / np.trace(Sw)

In [ ]:
J

0.11538703760378899